In [ ]:
!pip install langchain langchain_openai openai langchain-pinecone pinecone-notebooks langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
from langchain.schema import Document
import json
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from langsmith import traceable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing Data and Setting API KEYS**

In [ ]:
demographics = pd.read_parquet('/content/drive/MyDrive/Data/demographics.parquet')
diagnoses = pd.read_parquet('/content/drive/MyDrive/Data/diagnoses.parquet')
medications = pd.read_parquet('/content/drive/MyDrive/Data/medications.parquet')
notes = pd.read_parquet('/content/drive/MyDrive/Data/notes.parquet')
orders = pd.read_parquet('/content/drive/MyDrive/Data/orders.parquet')
vitals = pd.read_parquet('/content/drive/MyDrive/Data/vitals.parquet')
ordersclassified = pd.read_csv('/content/drive/MyDrive/Data/orders_with_classification.csv')

In [ ]:
# Set the environment variable
os.environ["OPENAI_API_KEY"] = "API_KEY"

In [ ]:
os.environ["PINECONE_API_KEY"]="API_KEY"

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'API_KEY'

**Initializing Vector Store and Adding Notes to It**

In [ ]:
pc = Pinecone()

In [ ]:
import time

index_name = "estimating-ehr-costs"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
result_df = (
    notes.groupby("USACSKEY_hashed", as_index=False)
    .agg({
        "FacilityCode_hashed": "first",
        "NoteTextCombined": " ".join
    })
)

In [ ]:
result_df.shape

(9962, 3)

In [ ]:
result_df.head()

,USACSKEY_hashed,FacilityCode_hashed,NoteTextCombined
0,0007015fcf776b912a6e42d2288f00c0b2c5894aa7aeaf...,2b4da2f6b7d30484b4962a682c5390aa823537fb78176a...,EMERGENCY DEPARTMENT ENCOUNTER Pt Name: G...
1,001071c48d588a069e807dc196939aea773143207a3ad1...,9eeabd9a714b2a462e6fd844ef0ab3dd95e3f3f8734cdd...,Emergency Medicine Provider Evaluation ...
2,0010c6783758a1b1ab1ceb38617090fceefb98edaea239...,d29bfa8dfec6c8092a3e12342fce9cfc1124540a892779...,Emergency Medicine Physician Evaluation ...
3,0011849e5a240b07237cac84dd51821efa0107af6532b0...,c8ba1c77fbf9815c9da22c603789adac8fd4d38f8845df...,He started feeling poorly well doing work out...
4,0017d62d73957115a5046983aeeff07430faf81fe3906f...,aeb2ef0493ab863f1fd0c64bc5cc74b1447517e8aa1059...,[HOSP] [HOSP] EMERGENCY DEPARTMENT EMERGENCY ...


In [ ]:
notes_json = result_df.to_json(orient="records")

In [ ]:
notes_json[0:5000]

'[{"USACSKEY_hashed":"0007015fcf776b912a6e42d2288f00c0b2c5894aa7aeafae74a5866eb68a83a2","FacilityCode_hashed":"2b4da2f6b7d30484b4962a682c5390aa823537fb78176a1ba55c59b2af2ba99b","NoteTextCombined":"EMERGENCY DEPARTMENT ENCOUNTER      Pt Name: Golf Tr-Couchxyz  MRN: [ID]2011  Birthdate [DATE]  Date of evaluation: [DATE]  ED Provider: [STAFF] [STAFF], MD     CHIEF COMPLAINT       Chief Complaint   Patient presents with    Gun Shot Wound     Trauma team: multiple gsw         HISTORY OF PRESENT ILLNESS   (Location\\/Symptom, Timing\\/Onset, Context\\/Setting, Quality, Duration, Modifying Factors, Severity) Note limiting factors.   I wore appropriate PPE for the entirety of this encounter.      HPI    [PATIENT] [PATIENT] is a [AGE] y.o. male who presents to the emergency department as a trauma team activation in the trauma bay after reported GSW s.  Patient was reportedly found near the hospital walking to the hospital after reportedly being shot at a bar.  EMS arrived near the hospital and 

In [ ]:
notes_data = json.loads(notes_json)

In [ ]:
# Convert JSON entries to LangChain Document objects
documents = []
for entry in notes_data:
    doc = Document(
        page_content=entry["NoteTextCombined"],
        metadata={"VisitID": entry["USACSKEY_hashed"], "LocationID": entry["FacilityCode_hashed"]}
    )
    documents.append(doc)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        chunked_doc = Document(page_content=chunk, metadata=doc.metadata)
        chunked_documents.append(chunked_doc)

In [ ]:
result_df['NoteTextCombined'].apply(len).mean()

7182.293716121261

In [ ]:
chunked_documents[0]

Document(metadata={'VisitID': '0007015fcf776b912a6e42d2288f00c0b2c5894aa7aeafae74a5866eb68a83a2', 'LocationID': '2b4da2f6b7d30484b4962a682c5390aa823537fb78176a1ba55c59b2af2ba99b'}, page_content='EMERGENCY DEPARTMENT ENCOUNTER      Pt Name: Golf Tr-Couchxyz  MRN: [ID]2011  Birthdate [DATE]  Date of evaluation: [DATE]  ED Provider: [STAFF] [STAFF], MD     CHIEF COMPLAINT       Chief Complaint   Patient presents with    Gun Shot Wound     Trauma team: multiple gsw         HISTORY OF PRESENT ILLNESS   (Location/Symptom, Timing/Onset, Context/Setting, Quality, Duration, Modifying Factors, Severity) Note limiting factors.   I wore appropriate PPE for the entirety of this encounter.      HPI    [PATIENT] [PATIENT] is a [AGE] y.o. male who presents to the emergency department as a trauma team activation in the trauma bay after reported GSW s.  Patient was reportedly found near the hospital walking to the hospital after reportedly being shot at a bar.  EMS arrived near the hospital and transpor

In [ ]:
embeddings=OpenAIEmbeddings(openai_api_key="API_KEY")

In [ ]:
uuids = [str(uuid4()) for _ in range(len(chunked_documents))]

In [ ]:
index = pc.Index("estimating-ehr-costs")

In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store.add_documents(documents = chunked_documents, ids=uuids)

['62958f1f-6703-4aaf-9dbf-4f6262bbea2e',
 '0089b3ca-6482-4671-8ae8-3fb863c7d5e4',
 '7953059f-4d80-4c55-a990-70069f1de944',
 '07ac9e58-366f-4766-b5f7-dd81c2f3dc8d',
 '7fddbe29-2d2f-41f6-84c1-8a52f413b97c',
 '8f4aa0ff-e142-4c77-857d-6c4d57fa3dc4',
 '28e16351-9e83-41f5-ac85-5861564325df',
 '60471a68-5517-44de-ab75-e7a83dbe7a15',
 'dfa50083-f475-40b0-a83c-ad68ff886563',
 '65509224-d0ef-4256-afd9-99f8ba500038',
 'c20bb5f2-47d0-4355-ae36-805601d46ecb',
 '29d06b82-6711-4f8e-a044-83124c767cab',
 '617e1a6a-960c-4deb-948f-45089176be9f',
 '07f3e0d9-87f0-4eae-bb39-f69fe7205a92',
 'b7b30b95-b09d-42ae-ad27-f3ef13850032',
 '5cf90bbf-c12f-46dd-b355-3929a8d383e2',
 '4c3f3c9d-99e9-4066-8837-45479c83a222',
 'dc52ee18-bad2-4b11-96fd-fe16115980a6',
 '055f8824-3cf2-4d67-ae2a-63e8e40f5d71',
 '66222543-160b-4a97-8367-5e820bf231e2',
 '1d6cb79b-bb38-4e34-a2d4-b766714f894f',
 '3b0388cd-b17a-4b56-92d8-42a4300688e8',
 '2e4d981a-483f-453f-a443-b98fae3805bd',
 'f98be6e1-1b18-458f-ac6b-77eadaa3cc27',
 'bb434273-e997-

In [ ]:
index_stats = index.describe_index_stats()

In [ ]:
index_stats

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 83555}},
 'total_vector_count': 83555}

In [ ]:
len(chunked_documents)

83555

**Making Consistent Naming Conventions for Order Names**

In [ ]:
ordersclassified.head() #provided by client - Ed is a physician that tried to go through all the Order names and clean them

,Order,IMO Title,Ed's Term
0,YELLOW URINE NO ADDITIVE,Additive for enteral formula,Other
1,Xray Chest 2 View PA + Lateral,"X-ray of chest, PA and lateral views",CXR
2,Xray Chest 1 View,"X-ray of chest, single view",CXR
3,XRAY CHEST 1 VIEW,"X-ray of chest, single view",CXR
4,"XR,CHEST 2 VIEWS","X-ray of chest, two views",CXR


In [ ]:
orders.head()

,VisitKey_hashed,OrderName,OrderType
0,4549882121fed351efe295e296429d29eae782bb5bf685...,ECG 12-LEAD,EKG
1,aac2cd46aef8d313a54d2eb98108cd8119d0a555fce8b7...,ECG 12-LEAD,EKG
2,4a44604e55c3973f69e6a5db27eef6b115774e11af1424...,ECG 12-LEAD,EKG
3,7549dea42abb7d1723cdcaae209fb1125a72bd5c1e64f5...,ECG 12-LEAD,EKG
4,e3e9eb5a2d072082ca207da888ed5627d01ca91e4e2ca4...,ECG 12-LEAD,EKG


In [ ]:
orders.shape

(88936, 3)

In [ ]:
ordersclassified.nunique()

,0
Order,999
IMO Title,423
Ed's Term,105


In [ ]:
ordersclassified.shape

(999, 3)

In [ ]:
orders["OrderName"] = orders["OrderName"].astype(str).str.upper()
ordersclassified["Order"] = ordersclassified["Order"].astype(str).str.upper()

In [ ]:
merged_orders = pd.merge(orders, ordersclassified, left_on="OrderName", right_on="Order", how="left")

**Utilizing TFID Vectorizer and Random Forest to Predict Order Names with Inconsistent Naming Convention**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    ordersclassified["Order"], ordersclassified["Ed's Term"], test_size=0.2, random_state=42
)

# Create a pipeline
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),  # Convert text to numerical features
    ("classifier", RandomForestClassifier())  # Classifier
])

# Train the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [ ]:
# Evaluate on test data
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.76


In [ ]:
unmatched_orders = merged_orders[merged_orders["Ed's Term"].isna()]

**Predicting the "Clean" Order Name for the Messy Order Names that Didn't have Ed's Term**

In [ ]:
# Identify unmatched rows
unmatched_orders = merged_orders[merged_orders["Ed's Term"].isna()]

# Extract the 'Order' column for unmatched rows
X_unmatched = unmatched_orders["OrderName"]

# Predict missing 'Ed's Term' for unmatched rows
unmatched_orders["Predicted Ed's Term"] = pipeline.predict(X_unmatched)

# Add predictions back into the original DataFrame
merged_orders.loc[merged_orders["Ed's Term"].isna(), "Ed's Term"] = unmatched_orders["Predicted Ed's Term"]

<ipython-input-44-b6ec8b2d9785>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_orders["Predicted Ed's Term"] = pipeline.predict(X_unmatched)


In [ ]:
merged_orders.to_csv("/content/drive/MyDrive/Data/cleaned_orders.csv")